In [20]:
import pandas as pd
import numpy as np
import os
import joblib
import time
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit 

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [21]:
# Path düzeltmesi
base_path = os.path.dirname(os.getcwd())  # notebooks/ klasörü
data_path = os.path.join(base_path, "data", "processed", "diplomacy")
print(f"Base path: {base_path}")
print(f"Data path: {data_path}")

Base path: c:\Users\aadil\Desktop\YAP470\amazon-fake-review-detector\notebooks
Data path: c:\Users\aadil\Desktop\YAP470\amazon-fake-review-detector\notebooks\data\processed\diplomacy


## 📂 Veri Yükleme

In [22]:
categorical_features = ["speaker", "receiver", "season"]
col_types = {col: 'object' for col in categorical_features}

# ⚠️ GERİ DÖNÜŞ: train_processed.parquet (çalışan versiyon)
train_df = pd.read_parquet(os.path.join(data_path, "train_processed.parquet"))
train_df = train_df.astype(col_types)
val_df = pd.read_parquet(os.path.join(data_path, "val_processed.parquet"))
val_df = val_df.astype(col_types)
test_df = pd.read_parquet(os.path.join(data_path, "test_processed.parquet"))
test_df = test_df.astype(col_types)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
print(f"\n📊 Available columns: {len(train_df.columns)} columns")
print(f"Columns: {train_df.columns.tolist()}")

Train: 13132, Val: 1416, Test: 2741

📊 Available columns: 13 columns
Columns: ['game_id', 'speaker', 'receiver', 'message_text', 'sender_intention', 'game_score', 'game_score_delta', 'year', 'season', 'original_fold', 'target', 'cleaned_text', 'message_length']


In [23]:
X_train = train_df.drop('target', axis=1)
y_train = train_df['target']
X_val = val_df.drop('target', axis=1)
y_val = val_df['target']
X_test = test_df.drop('target', axis=1)
y_test = test_df['target']

print(f"Class distribution (train): {y_train.value_counts().to_dict()}")

Class distribution (train): {0: 12541, 1: 591}


## ⚙️ Preprocessing Pipeline

In [24]:
# ✅ Temel numeric features (train_processed.parquet'ta var olanlar)
numeric_features = ["game_score", "game_score_delta", "year", "message_length"]

numeric_transformer = Pipeline(steps=[
  ("imputer", SimpleImputer(strategy="most_frequent")),
  ("scaler", StandardScaler())
])

categorical_features = ["speaker", "receiver", "season"]
categorical_transformer = Pipeline(steps=[
  ("imputer", SimpleImputer(strategy="constant", fill_value="most_frequent")),
  ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

text_feature = "cleaned_text"
text_transformer = TfidfVectorizer()

preprocessor = ColumnTransformer(transformers=[
  ('text', text_transformer, text_feature),
  ('num', numeric_transformer, numeric_features),
  ('cat', categorical_transformer, categorical_features)
], remainder="drop")

print(f"✅ Toplam numeric features: {len(numeric_features)}")

✅ Toplam numeric features: 4


In [25]:
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])
split_index = [-1] * len(X_train) + [0] * len(X_val)
ps = PredefinedSplit(test_fold=split_index)

## 🚀 Model 1: XGBoost + Class Weight Balancing

XGBoost'un avantajları:
- `scale_pos_weight`: Class imbalance için özel parametre
- L1/L2 regularization: Overfitting'i engelliyor
- Tree pruning: Daha optimize ağaç yapısı

In [26]:
# Class imbalance oranını hesapla
n_negative = (y_train == 0).sum()
n_positive = (y_train == 1).sum()
scale_pos_weight = n_negative / n_positive

print(f"Truth samples: {n_negative}")
print(f"Lie samples: {n_positive}")
print(f"Scale pos weight: {scale_pos_weight:.2f}")

Truth samples: 12541
Lie samples: 591
Scale pos weight: 21.22


In [27]:
pipeline_xgb_weighted = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBClassifier(
        random_state=42,
        scale_pos_weight=scale_pos_weight,  # Class imbalance için
        use_label_encoder=False,
        eval_metric='logloss'
    ))
])

In [31]:
# ⚡ HIZLI PARAMETRE GRİDİ (5-10 dakika)
param_grid_xgb = {
    'preprocessor__text__max_features': [3000],  # Sadece 1 seçenek
    'preprocessor__text__ngram_range': [(1, 2)],  # Bigram (daha iyi)
    'model__n_estimators': [200],  # Sabit
    'model__learning_rate': [0.05, 0.1],  # 2 seçenek
    'model__max_depth': [5, 7],  # 2 seçenek (9 çok yavaş)
    'model__scale_pos_weight': [15, 20],  # 2 seçenek (en kritik!)
    'model__subsample': [0.8],  # Sabit
    'model__colsample_bytree': [0.8],  # Sabit
    'model__min_child_weight': [1],  # Sabit
    'model__reg_alpha': [0],  # Regularization iptal
    'model__reg_lambda': [1]   # Sabit
}

# Kombinasyon sayısı hesapla
total_combinations = (
    len(param_grid_xgb['preprocessor__text__max_features']) *
    len(param_grid_xgb['preprocessor__text__ngram_range']) *
    len(param_grid_xgb['model__n_estimators']) *
    len(param_grid_xgb['model__learning_rate']) *
    len(param_grid_xgb['model__max_depth']) *
    len(param_grid_xgb['model__scale_pos_weight']) *
    len(param_grid_xgb['model__subsample']) *
    len(param_grid_xgb['model__colsample_bytree']) *
    len(param_grid_xgb['model__min_child_weight']) *
    len(param_grid_xgb['model__reg_alpha']) *
    len(param_grid_xgb['model__reg_lambda'])
)

print(f"⚡ HIZLI MOD: Toplam kombinasyon: {total_combinations}")
print(f"⏱️ Tahmini süre: {total_combinations * 0.5:.1f} - {total_combinations * 1:.1f} dakika")
print(f"\n📊 Değişiklikler:")
print(f"  ✅ 5,184 kombinasyon → {total_combinations} kombinasyon (%-{100 - total_combinations/5184*100:.0f} azalma!)")
print(f"  ✅ Sadece EN ÖNEMLİ parametreler: scale_pos_weight, learning_rate, max_depth")
print(f"  ✅ Bigram kullanılıyor (daha iyi text features)")
print(f"  ⚡ 90 dakika → 5-10 dakika!")

⚡ HIZLI MOD: Toplam kombinasyon: 8
⏱️ Tahmini süre: 4.0 - 8.0 dakika

📊 Değişiklikler:
  ✅ 5,184 kombinasyon → 8 kombinasyon (%-100 azalma!)
  ✅ Sadece EN ÖNEMLİ parametreler: scale_pos_weight, learning_rate, max_depth
  ✅ Bigram kullanılıyor (daha iyi text features)
  ⚡ 90 dakika → 5-10 dakika!


## ⚡ **HIZLI MOD: Parametre Sayısını Azalttık**

### **Problem:**
- **Önceki:** 5,184 kombinasyon → 90+ dakika 😰
- **Yeni:** 8 kombinasyon → **5-10 dakika** ⚡

### **Neyi Değiştirdik?**

#### **Kaldırılanlar (Hız için):**
```python
❌ max_features: [3000, 5000] → [3000]  (tek değer)
❌ n_estimators: [200, 300] → [200]  (200 yeterli)
❌ scale_pos_weight: [15, 20, 25] → [15, 20]  (2 yeter)
❌ subsample: [0.8, 0.9] → [0.8]  (sabit)
❌ colsample_bytree: [0.8, 1.0] → [0.8]  (sabit)
❌ min_child_weight: [1, 3] → [1]  (sabit)
❌ reg_alpha: [0, 0.1] → [0]  (regularization iptal)
❌ reg_lambda: [1, 5] → [1]  (sabit)
❌ max_depth: [5, 7, 9] → [5, 7]  (9 çok yavaş)
```

#### **Tutulanlar (En Önemliler!):**
```python
✅ scale_pos_weight: [15, 20]  ← EN KRİTİK! (Lie detection)
✅ learning_rate: [0.05, 0.1]  ← Önemli
✅ max_depth: [5, 7]  ← Önemli
✅ ngram_range: [(1, 2)]  ← Bigram (daha iyi)
```

### **Sonuç:**
- **8 kombinasyon** (5,184'ten %-99.8 azalma!)
- **5-10 dakika** sürecek
- En önemli 3 parametre korundu
- Performans kaybı minimal olacak (F1: ~0.24-0.26)

## 🎯 **İyileştirilmiş Parametre Grid**

### **Değişiklikler:**
1. **scale_pos_weight: [15, 20, 25]** - Daha agresif (önceki: 8-12)
2. **max_depth: [5, 7, 9]** - Daha derin ağaçlar (contradiction patterns için)
3. **colsample_bytree: [0.8, 1.0]** - Feature sampling (overfitting önler)
4. **min_child_weight: [1, 3]** - Leaf size control

**Neden Bu Parametreler?**
- Contradiction features karmaşık patterns → Daha derin ağaçlar gerek
- Class imbalance ciddi → scale_pos_weight artırılmalı
- Daha fazla feature var (13 → 22) → colsample_bytree ekledik

In [32]:
grid_search = GridSearchCV(
    pipeline_xgb_weighted, 
    param_grid_xgb, 
    cv=ps, 
    scoring='f1_macro',  # Lie class için daha adil
    n_jobs=4, 
    verbose=2
)

print("XGBoost GridSearchCV başlıyor...")
print(f"Toplam kombinasyon sayısı: {len(param_grid_xgb['preprocessor__text__max_features']) * len(param_grid_xgb['model__n_estimators']) * len(param_grid_xgb['model__learning_rate']) * len(param_grid_xgb['model__max_depth']) * len(param_grid_xgb['model__reg_alpha']) * len(param_grid_xgb['model__reg_lambda'])}")

XGBoost GridSearchCV başlıyor...
Toplam kombinasyon sayısı: 4


In [33]:
start_time = time.time()
grid_search.fit(X_train_val, y_train_val)
end_time = time.time()

print(f"\nEğitim süresi: {(end_time - start_time) / 60:.2f} dakika")
print(f"En iyi CV skoru (F1 Macro): {grid_search.best_score_:.4f}")
print(f"En iyi parametreler: {grid_search.best_params_}")

Fitting 1 folds for each of 8 candidates, totalling 8 fits


c:\Users\aadil\anaconda3\envs\ml\lib\site-packages\xgboost\core.py:158: UserWarning: [23:05:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Eğitim süresi: 0.82 dakika
En iyi CV skoru (F1 Macro): 0.5452
En iyi parametreler: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.05, 'model__max_depth': 5, 'model__min_child_weight': 1, 'model__n_estimators': 200, 'model__reg_alpha': 0, 'model__reg_lambda': 1, 'model__scale_pos_weight': 15, 'model__subsample': 0.8, 'preprocessor__text__max_features': 3000, 'preprocessor__text__ngram_range': (1, 2)}


In [34]:
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test)

print("\n" + "="*60)
print("TEST SETİ PERFORMANSI - XGBoost (Weighted)")
print("="*60)
print(classification_report(y_test, y_pred_test, target_names=['Truth (0)', 'Lie (1)']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred_test)
print(f"True Negatives:  {cm[0][0]}  (Correctly identified truth)")
print(f"False Positives: {cm[0][1]}  (Truth marked as lie)")
print(f"False Negatives: {cm[1][0]}  (Lie marked as truth)")
print(f"True Positives:  {cm[1][1]}  (Correctly identified lie)")


TEST SETİ PERFORMANSI - XGBoost (Weighted)
              precision    recall  f1-score   support

   Truth (0)       0.92      0.92      0.92      2501
     Lie (1)       0.17      0.17      0.17       240

    accuracy                           0.86      2741
   macro avg       0.55      0.55      0.55      2741
weighted avg       0.86      0.86      0.86      2741


Confusion Matrix:
True Negatives:  2305  (Correctly identified truth)
False Positives: 196  (Truth marked as lie)
False Negatives: 199  (Lie marked as truth)
True Positives:  41  (Correctly identified lie)


## 📊 **Güncellenmiş Parametre Sonuçları - Karşılaştırma**

### **🔄 İlk vs Güncellenmiş Sonuçlar:**

| Metrik | İlk Çalıştırma | Güncellenmiş | Değişim | Analiz |
|--------|----------------|--------------|---------|--------|
| **Lie F1** | 0.21 | **0.24** | +0.03 | ✅ %14 iyileşme |
| **Lie Precision** | 0.16 | **0.27** | +0.11 | ✅ %69 iyileşme! 🔥 |
| **Lie Recall** | 0.32 | **0.22** | -0.10 | ❌ %31 düştü |
| **Accuracy** | 0.79 | **0.88** | +0.09 | ✅ %11 iyileşme! |
| **F1 Macro** | 0.55 | **0.59** | +0.04 | ✅ %7 iyileşme |

---

### **🎯 Ne Oldu?**

#### **✅ BÜYÜK İYİLEŞME: Precision Patladı! (+69%)**

**İlk Çalıştırma:**
```
480 mesajı "yalan" dedi
    ✅ 77 tanesi doğruydu (16% precision)
    ❌ 403 tanesi yanlış alarm (84% hata!)
```

**Güncellenmiş:**
```
196 mesajı "yalan" dedi
    ✅ 53 tanesi doğruydu (27% precision) ← Neredeyse 2x daha iyi!
    ❌ 143 tanesi yanlış alarm (73% hata)
```

**Anlamı:** Model artık daha **seçici** ve **akıllı**. Rastgele "yalan" demiyor!

---

#### **❌ TRADE-OFF: Recall Düştü (-31%)**

**İlk Çalıştırma:**
```
240 yalandan 77 tanesi bulundu (32% recall)
163 yalan kaçtı
```

**Güncellenmiş:**
```
240 yalandan 53 tanesi bulundu (22% recall)
187 yalan kaçtı ← 24 yalan daha kaçtı
```

**Anlamı:** Model daha **muhafazakar** oldu, emin olmadığı yalanları işaretlemiyor.

---

### **🤔 Hangi Parametreler Bu Değişimi Yaptı?**

#### **1. `scale_pos_weight`: 10.85 → [8, 10, 12] (Manual)**
```python
# İlk: scale_pos_weight = 10.85 (otomatik, sabit)
# Yeni: 'model__scale_pos_weight': [8, 10, 12]
```
**Etki:** GridSearch muhtemelen **8** seçti (daha dengeli)
- 8 → Daha az agresif, precision ↑ recall ↓
- 12 → Daha agresif, precision ↓ recall ↑

---

#### **2. `subsample`: 1.0 → [0.8, 1.0]**
```python
'model__subsample': [0.8, 1.0]
```
**Etki:** Muhtemelen **0.8** seçildi
- Her ağaç %80 veri ile eğitildi (bagging effect)
- Overfitting azaldı
- Daha **genel** kalıplar öğrendi (noise'ı atlattı)

---

#### **3. Daha Fazla Kombinasyon**
Daha fazla parametre denendi → Daha iyi sweet spot bulundu

---

### **📈 Confusion Matrix Derinlemesine Analiz:**

```
                    İlk         Güncellenmiş    Fark
True Negatives:    2098         2358          +260 ✅ (Daha az false alarm!)
False Positives:    403          143          -260 ✅ (BÜYÜK İYİLEŞME!)
False Negatives:    163          187           +24 ❌ (Biraz daha yalan kaçtı)
True Positives:      77           53           -24 ❌ (Biraz daha az yalan yakalandı)
```

**Kritik:** 
- **260 false positive azaldı!** (403 → 143) 🎉
- Bunun bedeli: 24 yalan daha kaçtı (163 → 187)
- **Trade-off oranı:** 10:1 (Her 10 false positive azaltımı için 1 yalan kaçıyor)

---

### **💡 Hangisi Daha İyi?**

#### **Senaryo 1: Kullanıcı Sistemi (Önemli)**
```
False Positive: Kullanıcı rahatsız olur ❌
False Negative: Kullanıcı fark etmez ✅

Tercih: Güncellenmiş Model! 🏆
Sebep: 260 false alarm azaltımı çok değerli!
```

#### **Senaryo 2: Güvenlik/Spam Tespiti (Kritik)**
```
False Positive: Tolere edilebilir ✅
False Negative: Yalanı kaçırma tehlikeli ❌

Tercih: İlk Model
Sebep: Daha fazla yalan yakalıyor (77 vs 53)
```

#### **Senaryo 3: Akademik Proje (Dengeli)**
```
F1 Macro (genel performans):
İlk:          0.55
Güncellenmiş: 0.59 (+7%)

Tercih: Güncellenmiş Model! 🏆
Sebep: Daha dengeli, daha yüksek macro avg
```

---

### **🚀 Şimdi Ne Yapmalı?**

#### **Seçenek 1: Daha Fazla Recall İstiyorsanız** ⭐⭐⭐⭐
```python
# scale_pos_weight'i artırın:
'model__scale_pos_weight': [12, 15, 20]
```
**Beklenti:** 
- Recall: 0.22 → 0.30
- Precision: 0.27 → 0.20
- Daha fazla yalan yakalar ama false alarm artar

---

#### **Seçenek 2: Threshold Optimization** ⭐⭐⭐⭐⭐ (ÖNERİLEN!)
```python
# Farklı threshold'lar deneyin:
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Daha hassas (daha fazla yalan yakalar):
y_pred_03 = (y_pred_proba > 0.3).astype(int)

# Daha seçici (daha az false alarm):
y_pred_06 = (y_pred_proba > 0.6).astype(int)
```
**Avantaj:** Model'i yeniden eğitmeden precision/recall dengesini ayarlayabilirsiniz!

---

#### **Seçenek 3: train_final.parquet** ⭐⭐⭐⭐⭐ (EN İYİ!)
```python
# +9 contradiction features ekleyin:
train_df = pd.read_parquet("train_final.parquet")
```
**Beklenti:**
- Lie F1: 0.24 → **0.35-0.40**
- Hem precision hem recall artar!

---

### **📝 Özet:**

**✅ Güncelleme Başarılı!**
- Precision: +69% (0.16 → 0.27) 🔥
- Accuracy: +11% (0.79 → 0.88) 🔥
- F1 Macro: +7% (0.55 → 0.59) ✅
- False alarm: -65% (403 → 143) 🎉

**❌ Trade-off:**
- Recall: -31% (0.32 → 0.22)
- 24 yalan daha kaçtı

**🎯 Sonuç:** 
Model artık **daha akıllı ve dengeli**. Akademik proje için mükemmel!

**🚀 Sıradaki Adım Önerim:**
1. **Threshold optimization** deneyin (5 dakika, modeli yeniden eğitmeden)
2. **train_final.parquet** kullanın (+9 features)
3. Her ikisi de F1'i **0.35-0.40'a** çıkarabilir!

Hangisini denemek istersiniz? 🎯

## 💾 Model Kaydetme

In [ ]:
model_dir = os.path.join(os.path.dirname(base_path), "models", "diplomacy")
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, "xgboost_weighted.joblib")
joblib.dump(best_model, model_path)
print(f"Model kaydedildi: {model_path}")

## 📊 LightGBM ile Karşılaştırma

| Metrik | LightGBM | XGBoost | Değişim |
|--------|----------|---------|----------|
| Lie F1 | 0.19 | ?.?? | +?.?? |
| Lie Precision | 0.17 | ?.?? | +?.?? |
| Lie Recall | 0.23 | ?.?? | +?.?? |
| Accuracy | 0.84 | ?.?? | +?.?? |

**Sonuç çalıştırıldıktan sonra doldurun!**

## 🔍 **XGBoost Sonuç Analizi**

### **📊 Performans Özeti:**

| Metrik | LightGBM | XGBoost | Değişim | Yorum |
|--------|----------|---------|----------|-------|
| **Lie F1** | 0.19 | **0.21** | +0.02 | ✅ %11 iyileşme |
| **Lie Precision** | 0.17 | **0.16** | -0.01 | ❌ Biraz düştü |
| **Lie Recall** | 0.23 | **0.32** | +0.09 | ✅ %39 iyileşme! 🔥 |
| **Accuracy** | 0.84 | 0.79 | -0.05 | ⚠️ Düştü (trade-off) |
| **F1 Macro** | 0.55 | **0.55** | 0.00 | ➡️ Aynı |

---

### **🎯 Ne Değişti?**

#### **✅ İYİLEŞMELER:**

**1. Recall'da Büyük Artış (+39%):**
```
LightGBM: 240 yalandan 55 tanesi bulundu (%23)
XGBoost:  240 yalandan 77 tanesi bulundu (%32) ← +22 yalan daha!
```
**Anlamı:** XGBoost daha fazla yalanı yakalıyor! 🎉

**2. Confusion Matrix Karşılaştırması:**
```
                  LightGBM    XGBoost    Fark
True Positives:      55         77      +22 ✅ (Daha fazla yalan yakalandı)
False Negatives:    185        163      -22 ✅ (Daha az yalan kaçtı)
True Negatives:    2249       2098     -151 ❌ (Bazı doğrular yanlış etiketlendi)
False Positives:    252        403     +151 ❌ (Daha fazla false alarm)
```

---

#### **❌ KÖTÜLEŞMELER:**

**1. Precision Düştü (0.17 → 0.16):**
```
XGBoost 480 mesajı "yalan" dedi:
    ✅ 77 tanesi gerçekten yalan (16%)
    ❌ 403 tanesi aslında doğru (84%)  ← False alarm fazla!
```

**2. Accuracy Düştü (0.84 → 0.79):**
- Model daha "şüpheci" oldu
- Daha fazla mesajı yalan diye işaretledi
- Genel doğruluk %5 azaldı

---

### **💡 Neden Böyle Oldu?**

**XGBoost Stratejisi:**
```python
scale_pos_weight = 10.85  # Yalan sınıfına 10x daha fazla ağırlık
```
→ Model yalanları bulmaya daha fazla odaklandı  
→ Bunun için bazı doğruları feda etti (trade-off)

**LightGBM Stratejisi:**
```python
class_weight='balanced'  # Daha dengeli yaklaşım
```
→ Genel accuracy'yi korumaya çalıştı  
→ Ama yalanları kaçırdı

---

### **🤔 Hangisi Daha İyi?**

**Bağlama Göre Değişir:**

#### **Senaryo 1: Spam/Sahte Haber Tespiti (Kritik)**
```
❌ False Negative (Yalanı kaçırma): ÇOK KÖTÜ! 
✅ False Positive (Yanlış alarm):   Kabul edilebilir

Tercih: XGBoost ✅
Sebep: 22 yalan daha yakaladı, bu kritik!
```

#### **Senaryo 2: Kullanıcı Deneyimi (Önemli)**
```
❌ False Positive (Yanlış alarm):  KÖTÜ! (Kullanıcı rahatsız olur)
✅ False Negative (Yalanı kaçırma): Tolere edilebilir

Tercih: LightGBM ✅
Sebep: Daha az false alarm (252 vs 403)
```

#### **Senaryo 3: Akademik/Dengeli (Bu proje!)**
```
🎯 F1 Macro: İki sınıfın ortalaması
LightGBM: 0.55
XGBoost:  0.55  ← AYNI!

Tercih: XGBoost ≈ LightGBM
Sebep: F1 Macro eşit, ama recall daha yüksek
```

---

### **🚀 Sonraki Adımlar:**

#### **Öncelik 1: Threshold Ayarlama** ⭐⭐⭐⭐⭐
```python
# Şu anki: 0.5 threshold (default)
y_pred = model.predict(X_test)  

# Dene: 0.3-0.4 threshold
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.35).astype(int)  # Daha hassas
```
**Beklenti:** Precision artacak, recall biraz düşecek

---

#### **Öncelik 2: train_final.parquet Kullan** ⭐⭐⭐⭐⭐
```python
# Şu anki: train_processed.parquet (temel features)
# Değiştir: train_final.parquet (+9 contradiction features)
```
**Beklenti:** F1 (Lie) 0.21 → 0.30-0.35

---

#### **Öncelik 3: Ensemble** ⭐⭐⭐⭐
```python
ensemble = VotingClassifier([
    ('lgbm', lgbm_model),   # F1: 0.19
    ('xgb', xgb_model),     # F1: 0.21
    ('svm', svm_model)      # F1: 0.25 (arkadaşınız)
])
```
**Beklenti:** F1 (Lie) 0.21 → 0.28-0.32

---

### **📝 Özet:**

**✅ XGBoost'un Güçlü Yanları:**
- Recall: 0.32 (LightGBM'den %39 daha iyi) 🔥
- 22 yalan daha yakaladı
- Class imbalance'a daha agresif yaklaşım

**❌ XGBoost'un Zayıf Yanları:**
- Precision: 0.16 (çok düşük)
- 403 false positive (LightGBM'de 252)
- Accuracy: 0.79 (LightGBM'de 0.84)

**🎯 Karar:**
- **Akademik proje için:** XGBoost ≈ LightGBM (F1 Macro aynı)
- **Yalan tespiti öncelikse:** XGBoost ✅ (daha fazla yalan yakalar)
- **En iyi çözüm:** İkisini de ensemble'da kullan! 🚀

**💪 Ne Yapmalı:**
1. XGBoost + train_final.parquet dene (5 dakika)
2. Threshold optimization yap (10 dakika)
3. Ensemble yap (30 dakika)
4. Beklenen nihai F1 (Lie): **0.35-0.45**